In [195]:
import numpy as np
import cvxpy as cp
import pandas as pd
from scipy.spatial import distance
from sklearn import preprocessing
from scipy.cluster import hierarchy
from scipy.cluster import vq
import matplotlib.pyplot as plt
from datetime import date
from IPython.display import Image
import re
import datetime
%matplotlib inline


In [160]:
from pulp import *

In [156]:
prob = pulp.LpProblem('Diet', pulp.LpMinimize)
# decision variables
x1 = LpVariable('Therapists', 0, None, LpInteger)
x2 = LpVariable('Patients', 0, None, LpInteger)
x3 = LpVariable('Reviewers',0,None,LpInteger)
x4 = LpVariable('Transcribers',0,None,LpInteger)
# define patient time constraints
sessions = 1
hours_per_session = 3
totalPatientHrs = sessions * hours_per_session

# define therapist time constraints
totalTherapistHrs = 14

# objective function
prob += (totalTherapistHrs*x1 + totalPatientHrs*x2) , 'Total Time'

# constraints
prob += x1 + x2 >= 400, 'Required Patients'
prob += x1 == x2 
prob += x3 == 4
# solve
prob.solve()

1

In [157]:
print('Status:',LpStatus[prob.status])
for v in prob.variables():
    print(v.name,':',v.varValue)
print('Total Hours:', value(prob.objective))
print('Total Days:', value(prob.objective)/12)
print('Total Years:', value(prob.objective)/12/365)

Status: Optimal
Patients : 200.0
Reviewers : 4.0
Therapists : 200.0
Total Hours: 3400.0
Total Days: 283.3333333333333
Total Years: 0.776255707762557


In [288]:
# define impact of video transcription
vidHrs = 15
numTranscribers = 1
ratio = 0.75
revisedHrs = vidHrs*numTranscribers*ratio
revisedHrs

11.25

In [313]:
patients = 5000
region = []
days= []
DateOnline = []
vidHrs = np.ones(patients)*revisedHrs
StartDate = '11/30/19'
date1 = pd.to_datetime(StartDate)

for i in range(patients):
    region.append(int(np.random.uniform(1,10)))
    days.append(int(np.random.uniform(1,560)))
    DateOnline.append(date1 + pd.DateOffset(months=int(days[i]/30)))
trainees = pd.DataFrame({'Region':region,'Days':days,'DateOnline':DateOnline,'Video Hours':vidHrs}) 

print(trainees)   
trainee_pivot = trainees.pivot_table(margins=True,values='Days',aggfunc=len,index='DateOnline',columns='Region')
hours_pivot = trainees.pivot_table(margins=True,values='Video Hours',aggfunc=sum,index='DateOnline',columns='Region')


trainee_pivot.to_csv('trainees.csv')
hours_pivot.to_csv('video_hours.csv')
#data = pd.read_csv('trainees.csv')
#data[:5]

      Region  Days DateOnline  Video Hours
0          3   501 2021-03-30        11.25
1          3   143 2020-03-30        11.25
2          3   350 2020-10-30        11.25
3          2   398 2020-12-30        11.25
4          3   481 2021-03-30        11.25
5          8   297 2020-08-30        11.25
6          1    17 2019-11-30        11.25
7          9    80 2020-01-30        11.25
8          7   466 2021-02-28        11.25
9          8   402 2020-12-30        11.25
10         1   188 2020-05-30        11.25
11         9   550 2021-05-30        11.25
12         4   473 2021-02-28        11.25
13         3   482 2021-03-30        11.25
14         4   425 2021-01-30        11.25
15         9   506 2021-03-30        11.25
16         5   366 2020-11-30        11.25
17         5   133 2020-03-30        11.25
18         8    47 2019-12-30        11.25
19         7     8 2019-11-30        11.25
20         5   173 2020-04-30        11.25
21         2   141 2020-03-30        11.25
22         

In [345]:
trainees = pd.read_csv('trainees.csv')
trainees

,DateOnline,1,2,3,4,5,6,7,8,9,All
0,2019-11-30 00:00:00,30,33,34,34,39,31,36,31,31,299
1,2019-12-30 00:00:00,21,32,24,33,29,37,29,33,28,266
2,2020-01-30 00:00:00,30,33,29,31,33,28,36,24,30,274
3,2020-02-29 00:00:00,30,24,36,25,22,26,28,27,29,247
4,2020-03-30 00:00:00,23,24,30,34,37,29,37,35,36,285
5,2020-04-30 00:00:00,25,27,35,33,38,33,31,33,26,281
6,2020-05-30 00:00:00,23,30,27,25,28,30,29,31,27,250
7,2020-06-30 00:00:00,29,33,31,29,27,36,29,31,38,283
8,2020-07-30 00:00:00,22,39,34,26,25,42,36,28,37,289
9,2020-08-30 00:00:00,26,20,22,38,24,22,36,29,37,254


In [511]:
trainee_pivot

Region,1,2,3,4,5,6,7,8,9,All
DateOnline,,,,,,,,,,
2019-11-30 00:00:00,30,33,34,34,39,31,36,31,31,299
2019-12-30 00:00:00,21,32,24,33,29,37,29,33,28,266
2020-01-30 00:00:00,30,33,29,31,33,28,36,24,30,274
2020-02-29 00:00:00,30,24,36,25,22,26,28,27,29,247
2020-03-30 00:00:00,23,24,30,34,37,29,37,35,36,285
2020-04-30 00:00:00,25,27,35,33,38,33,31,33,26,281
2020-05-30 00:00:00,23,30,27,25,28,30,29,31,27,250
2020-06-30 00:00:00,29,33,31,29,27,36,29,31,38,283
2020-07-30 00:00:00,22,39,34,26,25,42,36,28,37,289


In [510]:
hours = pd.read_csv('video_hours.csv')
hours_pivot

Region,1,2,3,4,5,6,7,8,9,All
DateOnline,,,,,,,,,,
2019-11-30 00:00:00,337.50,371.25,382.50,382.50,438.75,348.75,405.00,348.75,348.75,3363.75
2019-12-30 00:00:00,236.25,360.00,270.00,371.25,326.25,416.25,326.25,371.25,315.00,2992.50
2020-01-30 00:00:00,337.50,371.25,326.25,348.75,371.25,315.00,405.00,270.00,337.50,3082.50
2020-02-29 00:00:00,337.50,270.00,405.00,281.25,247.50,292.50,315.00,303.75,326.25,2778.75
2020-03-30 00:00:00,258.75,270.00,337.50,382.50,416.25,326.25,416.25,393.75,405.00,3206.25
2020-04-30 00:00:00,281.25,303.75,393.75,371.25,427.50,371.25,348.75,371.25,292.50,3161.25
2020-05-30 00:00:00,258.75,337.50,303.75,281.25,315.00,337.50,326.25,348.75,303.75,2812.50
2020-06-30 00:00:00,326.25,371.25,348.75,326.25,303.75,405.00,326.25,348.75,427.50,3183.75
2020-07-30 00:00:00,247.50,438.75,382.50,292.50,281.25,472.50,405.00,315.00,416.25,3251.25


In [347]:
hours.index.get_level_values(0)

RangeIndex(start=0, stop=20, step=1)

In [348]:
hours.columns.get_level_values(0)

Index(['DateOnline', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'All'], dtype='object')

In [349]:
trainees.iloc[:]['All']

0      299
1      266
2      274
3      247
4      285
5      281
6      250
7      283
8      289
9      254
10     269
11     265
12     300
13     250
14     244
15     239
16     254
17     265
18     186
19    5000
Name: All, dtype: int64

In [376]:
len(hours.iloc[:]['All'])

20

In [325]:
# define how many review hrs are needed each month
numReviewers = 1
hrsMonth = 600
videoHrs = numReviewers*hrsMonth
print('video hours capacity per month:',videoHrs)

video hours capacity per month: 600


In [456]:
index = []
mthHrs = []
hrsDict = {}
for idx,rows in hours.iterrows():
    #print(rows['All'])
    index.append(idx)
    mthHrs.append(rows['All'])
    hrsDict[idx] = videoHrs - rows['All'] 
totalHrs = mthHrs[len(mthHrs)-1]
mthHrs = mthHrs[:-1]
print(index,mthHrs)
#print(totalHrs)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] [3363.75, 2992.5, 3082.5, 2778.75, 3206.25, 3161.25, 2812.5, 3183.75, 3251.25, 2857.5, 3026.25, 2981.25, 3375.0, 2812.5, 2745.0, 2688.75, 2857.5, 2981.25, 2092.5]


In [485]:
statusDict = {}
for key,value in hrsDict.items():
    if value < 0:
        statusDict[key] = 0
    else:
        statusDict[key] = 1
    #print(key,value)
statusDict

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0}

In [466]:
status = lambda x: videoHrs - x
vstat = np.vectorize(status)
statusMth = vstat(mthHrs)
statusMth

array([-2763.75, -2392.5 , -2482.5 , -2178.75, -2606.25, -2561.25,
       -2212.5 , -2583.75, -2651.25, -2257.5 , -2426.25, -2381.25,
       -2775.  , -2212.5 , -2145.  , -2088.75, -2257.5 , -2381.25,
       -1492.5 ])

In [442]:
status1 = status[status<0]
status1

array([-2763.75, -2392.5 , -2482.5 , -2178.75, -2606.25, -2561.25,
       -2212.5 , -2583.75, -2651.25, -2257.5 , -2426.25, -2381.25,
       -2775.  , -2212.5 , -2145.  , -2088.75, -2257.5 , -2381.25,
       -1492.5 ])

In [507]:
production1 = pulp.LpVariable.dicts('trainees',((idx,row['All']) for idx,row in trainees.iterrows()),lowBound=0,cat='Integer')

In [503]:
production1

{(0, 299): trainees_(0,_299),
 (1, 266): trainees_(1,_266),
 (2, 274): trainees_(2,_274),
 (3, 247): trainees_(3,_247),
 (4, 285): trainees_(4,_285),
 (5, 281): trainees_(5,_281),
 (6, 250): trainees_(6,_250),
 (7, 283): trainees_(7,_283),
 (8, 289): trainees_(8,_289),
 (9, 254): trainees_(9,_254),
 (10, 269): trainees_(10,_269),
 (11, 265): trainees_(11,_265),
 (12, 300): trainees_(12,_300),
 (13, 250): trainees_(13,_250),
 (14, 244): trainees_(14,_244),
 (15, 239): trainees_(15,_239),
 (16, 254): trainees_(16,_254),
 (17, 265): trainees_(17,_265),
 (18, 186): trainees_(18,_186),
 (19, 5000): trainees_(19,_5000)}

In [498]:
status1 = pulp.LpVariable.dicts('status',(statusDict[key] for key,value in statusDict.items()),cat='Binary')
status1

{0: status_0}

In [379]:
hrs = []
for i in range(len(hours.iloc[:]['All'])):
    hrs.append(hours.iloc[i]['All'])
hrs

[3363.75,
 2992.5,
 3082.5,
 2778.75,
 3206.25,
 3161.25,
 2812.5,
 3183.75,
 3251.25,
 2857.5,
 3026.25,
 2981.25,
 3375.0,
 2812.5,
 2745.0,
 2688.75,
 2857.5,
 2981.25,
 2092.5,
 56250.0]

In [512]:
model = pulp.LpProblem('mintime',pulp.LpMinimize)

In [ ]:
model += pulp.lpSum([production1[])

In [513]:
# define impact of video transcription
vidHrs = 15
numTranscribers = 1
ratio = 0.75
revisedHrs = vidHrs*numTranscribers*ratio
revisedHrs

11.25

In [540]:
patientDemand = np.ones(len(trainee_pivot.index))*patients/len(trainee_pivot.index)
# =========
therapists = 5000
patients = 2500
region = []
days= []
DateOnline = []
certified = []
vidHrs = np.ones(therapists)*revisedHrs
StartDate = '11/30/19'
date1 = pd.to_datetime(StartDate)

for i in range(therapists):
    region.append(int(np.random.uniform(1,10)))
    certified.append(np.random.choice([0,1],size=None))
    
    if certified[i] == 1:
        days.append(0)
        vidHrs[i] = 0
    else:
        days.append(int(np.random.uniform(1,560)))
    
    DateOnline.append(date1 + pd.DateOffset(months=int(days[i]/30)))
    

trainees = pd.DataFrame({'Region':region,'Certified':certified,'Days':days,'DateOnline':DateOnline,'Video Hours':vidHrs}) 
#print(certified)
#print(trainees)   
pool = trainees.pivot_table(margins=False,aggfunc=({'Days': 'sum','Video Hours':'sum'}), index=['DateOnline','Region'],columns='Certified')
poolCount = pd.crosstab([trainees.DateOnline,trainees.Region],trainees.Certified,margins=False)


trainee_pivot = trainees.pivot_table(margins=True,values='Days',aggfunc=len,index='DateOnline',columns='Region')
hours_pivot = trainees.pivot_table(margins=True,values='Video Hours',aggfunc=sum,index='DateOnline',columns='Region')



In [541]:
trainee_pivot.to_csv('trainees.csv')
hours_pivot.to_csv('video_hours.csv')
# ===========
pool.to_csv('pool.csv')
poolCount.to_csv('poolCount.csv')
pool_c = pd.read_csv('pool.csv')
poolCount_c = pd.read_csv('poolCount.csv')

In [548]:
pool.head()

Days      Video Hours     
Certified              0    1           0    1
DateOnline Region                             
2019-11-30 1       233.0  0.0      146.25  0.0
           2       233.0  0.0      180.00  0.0
           3       322.0  0.0      225.00  0.0
           4       176.0  0.0      123.75  0.0
           5       163.0  0.0      123.75  0.0

In [549]:
poolCount.head()

Certified           0    1
DateOnline Region         
2019-11-30 1       13  275
           2       16  263
           3       20  303
           4       11  288
           5       11  255

In [557]:
date = pool.index.get_level_values(0)
reg = pool.index.get_level_values(1)


In [620]:
d = []
regs = []
train = []
cert = []
daysTrain = []
daysCert = []
hrsTrain =[]
hrsCert = []
regions = 9
patients = 2500
for date,reg in pool.index:
    d.append(date)
    regs.append(reg)
    train.append(poolCount.loc[(date,reg),0])
    cert.append(poolCount.loc[(date,reg),1])
    daysTrain.append(pool.loc[(date,reg),('Days',0)])
    daysCert.append(pool.loc[(date,reg),('Days',1)])
    hrsTrain.append(pool.loc[(date,reg),('Video Hours',0)])
    hrsCert.append(pool.loc[(date,reg),('Video Hours',1)])
    #print(pool.loc[(date,reg),('Days',0)])
    #print(poolCount.loc[(date,reg),0])
    
df = pd.DataFrame({'Date':d,'Region':regs,'Train':train,'Cert':cert,'Days Train':daysTrain,'Days Cert':daysCert,'Hours Train':hrsTrain,'Hours Cert':hrsCert})
#df['video review capacity'] = (videoHrs/regions)*np.ones(len(df))
df['patients'] = int((patients/regions))*np.ones(len(df))
df['min train / cert']= np.minimum(df['Train'],df['Cert'])
df['cost'] = None
for i in df.index:
    print(df['Cert'][i])
    if df['min train / cert'][i] == df['Train'][i]:
        df['cost'][i] = df['Days Train'][i] + df['Hours Train'][i]
    else:
        df['cost'][i] = df['Days Cert'][i] + df['Hours Cert'][i]

275
263
303
288
255
294
298


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


294
272
0
0
0
0
0


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [622]:
df

,Date,Region,Train,Cert,Days Train,Days Cert,Hours Train,Hours Cert,patients,min train / cert,cost
0,2019-11-30,1,13,275,233.0,0.0,146.25,0.0,277.0,13,379.25
1,2019-11-30,2,16,263,233.0,0.0,180.00,0.0,277.0,16,413
2,2019-11-30,3,20,303,322.0,0.0,225.00,0.0,277.0,20,547
3,2019-11-30,4,11,288,176.0,0.0,123.75,0.0,277.0,11,299.75
4,2019-11-30,5,11,255,163.0,0.0,123.75,0.0,277.0,11,286.75
5,2019-11-30,6,17,294,224.0,0.0,191.25,0.0,277.0,17,415.25
6,2019-11-30,7,14,298,151.0,0.0,157.50,0.0,277.0,14,308.5
7,2019-11-30,8,12,294,188.0,0.0,135.00,0.0,277.0,12,323
8,2019-11-30,9,8,272,138.0,0.0,90.00,0.0,277.0,8,228
9,2019-12-30,1,16,0,743.0,NaN,180.00,NaN,277.0,0,NaN


In [643]:
df.pivot_table(values = ['Train','Cert','Days Train','Days Cert','Hours Train','Hours Cert','patients','min train / cert','cost'],index=['Date','Region'],aggfunc='max')

#['Train','Cert','Days Train','Days Cert','Hours Train','Hours Cert','patients','min train / cert','cost']

Cert  Days Cert  Days Train  Hours Cert  Hours Train  \
Date       Region                                                         
2019-11-30 1        275        0.0       233.0         0.0       146.25   
           2        263        0.0       233.0         0.0       180.00   
           3        303        0.0       322.0         0.0       225.00   
           4        288        0.0       176.0         0.0       123.75   
           5        255        0.0       163.0         0.0       123.75   
           6        294        0.0       224.0         0.0       191.25   
           7        298        0.0       151.0         0.0       157.50   
           8        294        0.0       188.0         0.0       135.00   
           9        272        0.0       138.0         0.0        90.00   
2019-12-30 1          0        NaN       743.0         NaN       180.00   
           2          0        NaN       739.0         NaN       191.25   
           3          0        NaN       561.0         NaN       135.00   
           4          0        NaN       673.0         NaN       180.00   
           5          0        NaN      1060.0         NaN       247.50   
           6          0        NaN       451.0         NaN       112.50   
           7          0        NaN       789.0         NaN       191.25   
           8          0        NaN       494.0         NaN       123.75   
           9          0        NaN       876.0         NaN       236.25   
2020-01-30 1          0        NaN      1260.0         NaN       191.25   
           2          0        NaN       994.0         NaN       146.25   
           3          0        NaN      1195.0         NaN       191.25   
           4          0        NaN      1207.0         NaN       180.00   
           5          0        NaN       899.0         NaN       135.00   
           6          0        NaN      1031.0         NaN       157.50   
           7          0        NaN      1275.0         NaN       202.50   
           8          0        NaN      1289.0         NaN       191.25   
           9          0        NaN       748.0         NaN       112.50   
2020-02-29 1          0        NaN      1854.0         NaN       202.50   
           2          0        NaN      1169.0         NaN       123.75   
           3          0        NaN      1933.0         NaN       202.50   
...                 ...        ...         ...         ...          ...   
2021-02-28 7          0        NaN      9259.0         NaN       225.00   
           8          0        NaN      7495.0         NaN       180.00   
           9          0        NaN      6503.0         NaN       157.50   
2021-03-30 1          0        NaN      5036.0         NaN       112.50   
           2          0        NaN      9897.0         NaN       225.00   
           3          0        NaN      2988.0         NaN        67.50   
           4          0        NaN      7914.0         NaN       180.00   
           5          0        NaN      5963.0         NaN       135.00   
           6          0        NaN      8894.0         NaN       202.50   
           7          0        NaN      6429.0         NaN       146.25   
           8          0        NaN      6956.0         NaN       157.50   
           9          0        NaN      6834.0         NaN       157.50   
2021-04-30 1          0        NaN      7883.0         NaN       168.75   
           2          0        NaN      6233.0         NaN       135.00   
           3          0        NaN      5756.0         NaN       123.75   
           4          0        NaN     11581.0         NaN       247.50   
           5          0        NaN      6825.0         NaN       146.25   
           6          0        NaN      8910.0         NaN       191.25   
           7          0        NaN      4141.0         NaN        90.00   
           8          0        NaN      4203.0         NaN        90.00   
           9          0        NaN      5260

In [694]:
print(df.index)
print()
print(df.columns)
df['Date'][0]
df['Region'][0]
df.set_index(['Date','Region'])
df[['Date','Region']]
df['Date'][1]

RangeIndex(start=0, stop=171, step=1)

Index(['Date', 'Region', 'Train', 'Cert', 'Days Train', 'Days Cert',
       'Hours Train', 'Hours Cert', 'patients', 'min train / cert', 'cost'],
      dtype='object')


Timestamp('2019-11-30 00:00:00')

In [691]:
#prod = pulp.LpVariable.dicts('prod',((df['Date'][date],df['Region'][reg]) for date,reg in df.index),lowBound=0,cat='Integer')


prod = pulp.LpVariable.dicts('prod',((d,reg) for df['Date'][d],df['Region'][reg] in df[['Date','Region']]),lowBound=0,cat='Integer')

d

ValueError: too many values to unpack (expected 2)

In [672]:
d

[Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-11-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-01-30 00:00:00'),
 Timestamp('2020-02-29 00:00:00'),
 Timestamp('2020-02-

In [695]:
cd

/Users/ed
